In [13]:
import os
import json
from sqlalchemy.orm import declarative_base
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker, declarative_base
import pandas as pd
import great_expectations as ge

In [7]:
file_path = '../Credentials/keys.json'
if os.path.exists(file_path):
    with open(file_path, 'r') as json_file:
        data = json.load(json_file)
        user = data["user"]
        password = data["password"]
        port = data["port"]
        server = data["server"]
        db = data["db"]
else:
    print(f"File '{file_path}' not found.")
db_connection = f"postgresql://{user}:{password}@{server}:{port}/{db}"
engine = create_engine(db_connection)
print(f"connected!")

connected!


In [10]:
engine = create_engine(db_connection) #We create a SQLAlchemy engine to connect to the database
connection = engine.connect() #We connect to the database
table_name = 'jobslinkedinsalary'  
df = pd.read_sql_table(table_name, connection) #We read the table into a pandas dataframe
LinkedinSalary=df

In [14]:
# Creamos un dataset de GX a partir del DataFrame
linkedin_gx=ge.from_pandas(LinkedinSalary)